In [1]:
import statsmodels.api as sm
import time
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame
# from keras import backend as K
from math import log
from sklearn import metrics
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.model_selection import train_test_split
# from keras.datasets import mnist
# from keras.models import Model  #采用通用模型
# from keras.layers import Dense, Input  #只用到全连接层
# from sklearn.metrics.cluster.supervised import contingency_matrix
from sklearn.metrics import accuracy_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_selection import VarianceThreshold
from math import log
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import statsmodels.api as sm
import warnings
import numpy as np
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
# sc.logging.print_header()
warnings.filterwarnings("ignore")
np.random.seed(1337)  # for reproducibility
start_time1 = time.time()

data_names = [
    'Kumar',
#     'KumarTCC',
#     'Wang',
#     'Wallrapp',
#     'Patel',
#     'Haber',
#     'Petropoulos',
#     'Klein',
#     'Han',
#     'Grun',
#     'Cao',
#     'Spallanzani',
#     'Zemmour',
#     'Sala',
#     'Shekhar',
]


In [ ]:
def del_zero(df, num=1, ratio=0.4):
    '''
    剔除单一值列和去掉过多的0值特征列
    input:
    df:原始数据帧
    num:单一值阈值
    ratio:零值比例，超过则剔除
    retrun:
    df:筛选过后的数据帧
    '''
    feature_list = []
    for col in df.columns:
        unique_num = len(df.loc[:, col].unique())
#         print(unique_num)
        zeros_ratio = (df.loc[:, col] == 0).sum() / len(df.loc[:, col])
        if unique_num > num and zeros_ratio < 0.4:
            feature_list.append(col)
    return df[feature_list]


def data_standardization(X):
    '''
    数据标准化函数
    return:
    X:未经过数据标准化的数据
    return:
    X:经过数据标准化后的数据
    '''
    X = np.array(X)
    std_model = MinMaxScaler()
    std_model.fit(X)
    X = std_model.transform(X)
    X = pd.DataFrame(X)
    for i in range(len(X.columns)):
        if len(X.iloc[:, i].unique()) in [label_sze, label_sze+1]:
            X.iloc[:, i] *= 50
    return X


def getEntropy(s, pl_n=6):
    '''
    分箱去噪
    input:
    s:一个特征数据，一维
    pl_n:箱子的阈值,少于这个阈值则去掉
    return:
    返回信息熵
    '''
    prt_ary = 0.0
    data_vc = s.value_counts()  #统计每个箱子样本数量
    data_vc = data_vc[data_vc > pl_n]  #加入箱子的样本数量小于pl_n就去掉
    Sum = data_vc.values.sum()
    #计算信息熵
    for key, value in zip(data_vc.index, data_vc.values):
        prt_ary += np.log2(value / Sum) * (value / Sum)
    return -prt_ary  #返回信息熵



def points_to_the_noise(df, bins = 20,std_ol=4):
    '''
    分箱去噪 and 异常值平滑
    input:
    df:原始数据帧
    bins:分箱个数
    std_ol:认为异常值的标准差倍数
    return:
    df:筛选过后或经过异常平滑的数据帧
    '''
    # 数据异常平滑
    data_ol = pd.DataFrame()
    for col in df.columns:
        ol = np.array(df.loc[:, col])
        ol_mean = ol.mean()
        ol_std = ol.std()
        ol[ol > ol_mean + std_ol * ol_std] = ol_mean + std_ol * ol_std
        ol[ol < ol_mean - std_ol * ol_std] = ol_mean - std_ol * ol_std
        df[col] = ol
    #分箱去噪
    feature_list = []
    for col in df.columns:  #data是DataFrame格式的数据，行是细胞样本  列是特征
        bin_data = pd.cut(df.loc[:, col], bins=bins, labels=False)
        comentropy = getEntropy(bin_data)
        feature_list.append(comentropy > 0.3)
    df = df.loc[:, feature_list]

    return df



#hsnw构图
import hnswlib
import time
import os
import psutil
def get_ann(length,dimen):
    #向量维度
    dim = dimen
    num_elements = length
    data = X
    data_labels = np.arange(num_elements)
#     k = int(0.1 * int(X.shape[0]))
    k = 4
    #print(data_labels)
    # 声明索引
    p = hnswlib.Index(space = 'cosine', dim = dim) # hnswlib支持的距离有L2距离，向量内积以及cosine相似度
    # 初始化index
    p.init_index(max_elements = num_elements, ef_construction = 100, M = 16)
    # ef: 动态检索链表的大小。ef必须设置的比检索最近邻的个数K大。ef取值范围为k到集合大小之间的任意值。
    p.set_ef(int(k))
    p.set_num_threads(4)#cpu多线程并行计算时所占用的线程数
    
    #构建items
    p.add_items(X, data_labels)
    index_path = 'data.bin'
    p.save_index(index_path)
    global labels, distances
    labels, distances = p.knn_query(data, k = k)   #分别包括k个最近邻结果的标签和与这k个标签的距离。
    return labels,distances



def creat_adjmatrix(new_labels):
    #构建邻接矩阵
    n1 = np.zeros((len(new_labels),len(new_labels)))   #获取一个样本*样本的全零数组
    #赋值，离中心节点近的点的坐标进行赋值
    len1 = len(new_labels[0])    
    for i in range(len(new_labels)):
        for j in range(0,len1):
            a=new_labels[i][j]
            n1[i][a] = distances[i][j]
    b = np.mat(n1)   #转成矩阵
    B = b
    return B
# B = creat_adjmatrix(new_label)

#判断正态分布函数
from scipy import stats
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

def decide_data(df, p_=0.05):
    '''
    df:数据表，DataFrame格式
    p_：判断条件，p越大要求越严格
    '''
    #创建两个列表，分别保存满足正态分布的特征和不满足正态分布的特征
    columns1 = []
    columns2 = []
#     print(df[1].dtype)
    for line in df.columns:
        #判断是否数值类型，如果不是数值类型则跳过
        if df[line].dtype in [ 'int64', 'float64','float32','int32'] :
            #判断是否满足正态分布
            if normal_test(df[line], p_=0.05):
                columns1.append(line)
            else:
                columns2.append(line)
        else:
            columns1.append(line)
            columns2.append(line)
    return df[columns1], df[columns2]

def normal_test(ser, p_=0.05):
    '''
    这是一个正太分布判断函数
    ser:传入的待判断数据，一维数组、列表等
    p_：判断条件，p越大要求越严格
    '''
    #转为DataFrame格式
    s = pd.DataFrame(np.array(ser),columns = ['value'])
    u = s['value'].mean()  # 计算均值
    std = s['value'].std()  # 计算标准差
    p = stats.kstest(s['value'], 'norm', (u, std))[-1] #得到p值
    #判断是否满足正态分布，满足返回True，否则返回False
    if p>p_:
        return True
    else:
        return False
    
# 定义3σ法则识别异常值函数
def three_sigma(df, std_=3):
    '''
    df：表示传入的DataFrame
    std_：表示做多少倍的标准差
    '''
    #定义列表，ol_list存放异常样本的索引，relu1_list存放上届，relu2_list存放下界
    ol_list = []
    relu1_list = []
    relu2_list = []
    #变量所有变量
    for line in df.columns:
        #判断是否数值类型，如果不是数值类型则跳过，否则进行异常值检测
        if df[line].dtype in [ 'int64', 'float64'] :
            #获取当前待检测变量
            Ser1 = df['{}'.format(line)]
            #上届限
            relu1 = Ser1.mean()+std_*Ser1.std()
            #下届限
            relu2 = Ser1.mean()-std_*Ser1.std()
            #获取异常数组
            rule = (relu2>Ser1) | (relu1< Ser1)
            #获取异常样本索引
            index = np.arange(Ser1.shape[0])[rule]
            #异常标注列
            df['{}_ol'.format(line)] = np.array(rule).astype('int64')
            #分别添加异常样本索引、上届、下界
            outrange = Ser1.iloc[index]
            ol_list += index.tolist()
            relu1_list.append(relu1)
            relu2_list.append(relu2)
        else:
            continue
    return df, ol_list, relu1_list, relu2_list

def box_plot(df, box_=1.5):
    '''
    df：表示传入的DataFrame
    box_:表示箱线倍数
    
    '''
    #定义列表，ol_list存放异常样本的索引，Low_list存放上限，Up_list存放下限
    ol_list = []
    Low_list = []
    Up_list = []
    #遍历所有的列
    for line in df.columns:
        #判断是否数值类型，如果不是数值类型则跳过，否则进行异常值检测
        if df[line].dtype in [ 'int64', 'float64'] :#and len(df['{}'.format(line)].unique())>1
            #拿到当前列
            Ser = df['{}'.format(line)]
            #计算上下限
            Low = Ser.quantile(0.25)-box_*(Ser.quantile(0.75)-Ser.quantile(0.25))
            Up = Ser.quantile(0.75)+box_*(Ser.quantile(0.75)-Ser.quantile(0.25))
            #获取异常索引
            index = (Ser< Low) | (Ser>Up)
            
            #异常标注列
            df['{}_ol'.format(line)] = np.array(index).astype('int64')
            rule = np.arange(Ser.shape[0])[index]
            #分别添加异常样本索引、上届、下界
            ol_list += rule.tolist()
            Low_list.append(Low)
            Up_list.append(Up)
        else:
            continue
    return df, ol_list, Low_list, Up_list


def DBSCAN_model(df, min_samples=5, eps_=1):
    '''
    这是单变量的DBSCAN异常检测模型
    df:传入DataFrame格式待检测表
    min_samples:最小成簇样本数
    eps_：密度可达距离
    '''
    #定义一个ol_list，用来存放出现异常的样本索引
    ol_list = []
    #遍历所有特征列
    for line in df.columns:
        #判断是否数组类型，如果不是数值类型则跳过，否则进行异常值检测
        if df[line].dtype in [ 'int64', 'float64','float32','int32'] :#and len(df['{}'.format(line)].unique())>1
            #使用DBSCAN进行异常值检测，传入min_samples和eps_
            dbscan = DBSCAN(min_samples=min_samples, eps=eps_)
            #标准化当前待检测的特征变量
            std_data = scale(np.array(df['{}'.format(line)]).reshape((-1, 1)))
            #进行聚类
            dbscan.fit(std_data)
            #判断聚成的类别数，如果类别数大于1，则表明出现了异常样本
            if len(pd.Series(dbscan.labels_).unique())>1:
                #在sklearn中，将出现异常值的类标记为-1，所以非-1类的都为正常样本标记为0
                dbscan.labels_[dbscan.labels_ != -1] = 0
                #出现异常的样本表记为1
                dbscan.labels_[dbscan.labels_ == -1] = 1
                #标记列命名格式为  原列名_ol
                df['{}_ol'.format(line)] = dbscan.labels_
            else:
                #没有出现异常样本，所有样本都标记为0，标记列命名格式为  原列名_ol
                df['{}_ol'.format(line)] = np.zeros(df.shape[0], dtype='int64')
            #将当前特征变量出现异常值的样本加入ol_list保存
            ol_index = df['{}_ol'.format(line)][df['{}_ol'.format(line)]==1].index.tolist()
            ol_list += ol_index
        else:
            continue
    #返回一个DataFrame表和出现异常的样本索引
    return df, ol_list 


def  Many_dimensions_DBSCAN(df, min_samples=5, eps_=1):
    '''
    这是多变量的DBSCAN异常检测模型
    df:传入DataFrame格式待检测表
    min_samples:最小成簇样本数
    eps_：密度可达距离
    '''
    #定义一个ol_list，用来存放出现异常的样本索引
    ol_list = []
    #义一个feature_extraction，用来做特征的选取，将需要做检测的特征放入该list
    feature_extraction = []
    for line in df.columns:
        #判断是否数组类型，如果不是数值类型或者字符类型取值个数小于6则跳过，否则进行异常值检测
        if df[line].dtype in [ 'int64', 'float64'] or  len(df[line].unique())<6:
            #判断是否字符变量
            if df[line].dtype=='object' and len(df[line].unique())<6:
                #如果是字符变量，则需要数值化
                lbe_model = LabelEncoder()
                df[line] = lbe_model.fit_transform(df[line])
            #添加当前变量到特征列表里
            feature_extraction.append(line)
    #选取特征
    df = df[feature_extraction]
    #如果只有一个特征，则需要将这个变量变成两维并标准化，sklearn要求输入两维
    if len(df.shape)<0:
        std_data = scale(np.array(df).reshape((-1,1)))
        print("数据表可用特征少于两维！")
        return None
    else:
        std_data = scale(np.array(df))
    #创建一个降维类，将数据降到两维
    pca=PCA(n_components=2)
    pca_data=pca.fit_transform(std_data)
    #创建一个DBSCAN类
    dbscan = DBSCAN(min_samples = min_samples, eps=eps_)
    #训练
    dbscan.fit(pca_data)
    #将降维后的数据转为DataFrame格式
    pca_data = pd.DataFrame(pca_data, columns=['y1', 'y2'])
    #预测类别
    clusters = dbscan.fit_predict(pca_data)
    #判断聚成的类别数，如果类别数大于1，则表明出现了异常样本
    if len(pd.Series(dbscan.labels_).unique())>1:
        #在sklearn中，将出现异常值的类标记为-1，所以非-1类的都为正常样本标记为0
        dbscan.labels_[dbscan.labels_ != -1] = 0
        dbscan.labels_[dbscan.labels_ == -1] = 1
        pca_data['outlier'] = dbscan.labels_
    else:
        #没有出现异常样本，所有样本都标记为0，标记列命名格式为  原列名_ol
        pca_data['outlier'] = np.zeros(df.shape[0], dtype='int64')
    ol_index = pca_data['outlier'][pca_data['outlier']==1].index.tolist()
    #返回一个DataFrame表和出现异常的样本索引
    return pca_data, ol_index 





#传入邻居信息
def ol_array(data):

    '''
    单变量检测
    input
    data：DataFrame格式的待检测二维表
    p_：正态分布判断条件，p越大要求越严格
    std_：认为异常点的标准差倍数
    box_：表示箱线位置
    return
    ol_list:异常点索引
    '''
    data1,data2 = decide_data(data.copy(), p_=0.05)
    #调用n倍σ单变量检测函数，传入数据表和σ倍数n
    sigma_ol, sigmaol_list, upper_limit, lower_limit = three_sigma(data1.copy(), std_=4)
    #去重
    sigmaol_list = list(set(sigmaol_list))

    #调用n倍σ单变量检测函数，传入数据表和σ倍数n
    box_ol, boxol_list, upper_limit, lower_limit = box_plot(data2.copy(), box_=2)
    #去重
    boxol_list = list(set(boxol_list))



    ol_list = list(set(sigmaol_list+boxol_list))
    return ol_list


#这里的data是一个DataFrame格式的矩阵

def similitude(a,b):
    '''
    求x和y的相似性,余弦
    '''
    if a.shape[0] == b.shape[0]:
        a = a.reshape((1, -1))
    a_norm = np.linalg.norm(a)
    b_norm = np.linalg.norm(b)
    cos = (np.dot(a,b)/(a_norm * b_norm)).astype(float)
#     cos = cos.astype(float)
    return cos
#     return np.sum((x-y)**2)**0.5

#距离
def diff_k(x_test, x_train, k=5):
    '''
    计算前k个最近邻点，这里用的是欧式距离
    '''
    diff = x_test-x_train
    squreDiff = diff**2
    squreDist = np.sum(squreDiff,axis=1)
    distances = squreDist ** 0.5
    nearest = np.argsort(distances) # 对距离排序并返回对应的索引
    nearest_index = nearest[:k]
    #获得前k个最近邻点的索引并返回
    return nearest_index

#根据余弦相似度返回索引
def simcos_k(a,b,k = 5):
    '''
    求x和y的相似性,余弦,返回其索引。
    '''
    if a.shape[0] == b.shape[0]:
        a = a.reshape((1, -1))
    a_norm = np.linalg.norm(a)
    b_norm = np.linalg.norm(b)
    cos = np.dot(a,b)/(a_norm * b_norm)
#     trainx = trainx.astype(float)
    nearset = np.argsort(cos)
    nearest_index = nearset[:k]
    return nearest_index






#图卷积自编码
import torch
from sklearn.decomposition import PCA
def loaddata(data_ol1,label):
    Features = np.array(data_ol1.T)
    Features = torch.Tensor(Features)
    label = np.array(label).reshape(len(label),1)
    Labels = label
    return(Features, Labels)


    
    
    
class myGAE(torch.nn.Module):
    def __init__(self, d_0, d_1):
        super(myGAE, self).__init__()

        self.gconv1 = torch.nn.Sequential(
            torch.nn.Linear(d_0, d_1),
            torch.nn.ReLU(inplace=True)
        )
        self.gconv1[0].weight.data = get_weight_initial(d_1, d_0)

        
    def Agg(self,data_ol,data):
        import math
        import torch
        #df 是做完领域聚合的结果
        data = pd.DataFrame(data.numpy())
        data_ol = pd.DataFrame(data_ol.numpy())
        df = pd.DataFrame()
        data_ol1 = pd.DataFrame(np.array(data_ol),columns=['{}'.format(i) for i in range(data_ol.shape[1])])
        data_ol2 = data_ol.T
        k=4
        cell_data = np.array(data_ol1)    #维度124,2000
        #遍历所有的样本点
        for cell in range(data.shape[1]):     #data.shape = 124,124
            #中心点
            df['x_{}'.format(cell)] = np.array(data.iloc[:,cell]).reshape(-1)
            #循环迭代更新特征
            while True:
                #x_test表示中心点,x_test表示中心点的那一列，维度（124，1）
                x_test = np.array(data.iloc[:,cell]).reshape((-1, 1))
                cell_test =np.array(data_ol2.iloc[:,cell]).reshape((-1, 1))  #维度2000，1
                #x_train表示待求中心点最近邻数据集，去除中心点的其余所有列（124，123）
                x_train = np.array(data.drop(labels = data.columns[cell], axis=1))
                x_train1 = np.array(data)
                #获得最近邻索引  根据欧氏距离建立索引，排序取前三个，因为k为3
                nearest_index = new_labels[cell]  
                #获得k个样本点的特征，相似性，维度为（3，124）
                x_k = pd.DataFrame(x_train1[:, nearest_index])
#                 #获得邻域异常点索引
                ol_index = ol_array(x_k.T)
                #原始细胞特征  维度为（12,2000）
                cell_k = pd.DataFrame(cell_data[nearest_index,:])  #维度12，2000
                cell_k = cell_k.T
# #                 删除异常值索引
                cell_k = cell_k.drop(labels = cell_k.columns[ol_index],axis = 1)
                cell_k = cell_k.T
                cell_k = np.array(cell_k)
                #设定相容类阈值和其他一些参数
                d = 0.95
                sims = 0
                simuv = 0
                #循环中心点和最近邻点求相容类
                for i in range(x_k.shape[1]):
                    share = 0
                    count = 0
                    simh = 0
                    agg =0
                    v = i+1
                    for j in range(i, x_k.shape[1]):
                        cell_j = np.array(cell_k[j,:]).reshape(-1, 1)
                        cell_v = np.array(cell_k[v,:]).reshape(-1, 1)
                        #求共享邻居
                        nearest_index1 = new_labels[i]
                        nearest_index2 = new_labels[j]

                        if similitude(cell_v,cell_j)>d:
                            #定义一：求相容类的个数
                            count = count + 1   
                            #求两个交集即共享邻居
                            share=len(list(set(nearest_index1).intersection(set(nearest_index2))))
                            #定义三：引入共享近邻的邻域节点与中心节点相似度
                            sims = (2 * share) / (len(set(nearest_index1)) + len(set(nearest_index2)))


                    #定义二：通过邻居节点集的信息熵定义节点u的领域相似度为Sim(г(u))。  
                    simh += -(count/k * np.log(count/k))
                    #定义四：结合以上两个部分的相似性度量，邻域节点与中心节点之间相似性度量定义如下：simuv = a*sims +b*simh  a+b=1
                    a,b,z= 0,0,0
                    a = 0.5
                    b = 0.5
                    simuv = a*sims + b*simh
#                     print('simuv',simuv)
#                     print('sims',sims)
#                     print('simh',simh)
                    z = (1 - simuv)
                    C = data_ol1.values
                    D = B*C
                    E = pd.DataFrame(D)
                    for m in range(0,cell_k.shape[0]-1):
                        if(0.3<z<0.8):
                             #找邻居的邻居点索引
#                             print('z1',z)
                            near_index1 = new_labels[cell]           #一阶邻居索引
                            near_index2 = new_labels[near_index1]    #二阶邻居索引
                            data_ol01 = data_ol1.iloc[near_index1]    #一阶邻居索引所得特征
                            data_ol02 = data_ol1.iloc[near_index2[1]]    #二阶邻居索引所得特征
                            n0 = np.ones((1,k))                     #生成1*k的邻接矩阵
                            n0 = n0 / k
                            n01 =pd.DataFrame(n0).to_numpy()
                            n02 = np.dot(n01,data_ol02)                  #得到1*k*k*特征数的矩阵   如1*2000这样
                            data_ol11 = data_ol1.values                  #data_ol11为数组类型，目的是进行聚合
                            n2 = data_ol11[cell] + n02               #得到聚合之后的一行    一个样本和他的特征
                            data_ol11[cell] = n2                    #把得到聚合后的加给该中心节点
                            c1 = data_ol11
                            d1 = B*c1
                            e1 = pd.DataFrame(d1)
                            if data_ol1.iloc[i,m] !=0:
                                    data_ol1.iloc[i,m] = e1.iloc[i,m]
                                
                        if(z>0.8):
#                             print('z2',z)
                            if data_ol1.iloc[i,m] !=0:
                                data_ol1.iloc[i,m] = E.iloc[i,m]      #一阶聚合
                        if(z<0.3):
#                             print('z3',z)
                            break
                    break
                break
            break


        my_array = np.array(data_ol1)
        my_tensor = torch.tensor(my_array)
        return(my_tensor)

    def Encoder(self, Adjacency_Modified, H_0):
        H_0 = self.Agg(H_0, Adjacency_Modified)
        H_1 = self.gconv1(torch.matmul(Adjacency_Modified, H_0))
#         H_2 = self.gconv2(torch.matmul(Adjacency_Modified, H_1))
        return H_1

    def Graph_Decoder(self, H_1):
        graph_re = Graph_Construction(H_1)
        Graph_Reconstruction = graph_re.Middle()
        return Graph_Reconstruction


    def forward(self, Adjacency_Modified, H_0):
        Latent_Representation = self.Encoder(Adjacency_Modified, H_0)
        Graph_Reconstruction = self.Graph_Decoder(Latent_Representation)
        return Graph_Reconstruction, Latent_Representation
    

    
    
def cluster(X, y, label_sze):
    ari = nmi = 0
    for n_clusters in range(2, 2 * label_sze + 1):
        clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(X)
        ari_ = metrics.adjusted_rand_score(y.reshape((-1)), clustering.labels_)
        nmi_ = metrics.adjusted_mutual_info_score(y.reshape((-1)),
                                                  clustering.labels_)
        if (ari_ + nmi_) > (ari + nmi):
            ari, nmi = ari_, nmi_
    return ari, nmi



    ################################### Calculate the adjacency matrix #########################################################
def graph_autoencoder(Features, Labels, Adjacency_Matrix_raw,label):
    Features = torch.Tensor(Features)
    Adjacency_Matrix = torch.Tensor(Adjacency_Matrix_raw)
    label1 = label
    ################################################ adjacency convolution ##################################################
    convolution_kernel = Convolution_Kernel(Adjacency_Matrix)
    Adjacency_Convolution = convolution_kernel.Adjacency_Convolution()
    ############################################ Results  Initialization ###################################################
    ACC_GAE_total = []
    NMI_GAE_total = []
    PUR_GAE_total = []
    ARI_GAE_total = []

    ACC_GAE_total_STD = []
    NMI_GAE_total_STD = []
    PUR_GAE_total_STD = []
    ARI_GAE_total_STD = []

    #######################################  Model #########################################################################
    bce_loss = torch.nn.BCELoss(size_average=True)
    model_GAE = myGAE(Features.shape[1], Hidden_Layer_1)
    optimzer = torch.optim.Adam(model_GAE.parameters(), lr=Learning_Rate)
    start_time = time.time()

    # cel_loss=torch.nn.CrossEntropyLoss()
    # input=torch.Tensor([[-0.7715, -0.6205,-0.2562]])
    # target = torch.tensor([0])
    # output = entroy(input, target)
    # print(output)

    #######################################  Train and result ################################################################
    for epoch in range(Epoch_Num):

        Graph_Reconstruction, Latent_Representation = model_GAE(Adjacency_Convolution, Features)
        Graph_Reconstruction=torch.unsqueeze(Graph_Reconstruction,0)
    #     print(Graph_Reconstruction.view(-1).shape)
    #     print(Adjacency_Matrix.view(-1).shape)
        loss = bce_loss(Graph_Reconstruction.view(-1), Adjacency_Matrix.view(-1))

        optimzer.zero_grad()
        loss.backward()
        optimzer.step()

        Latent_Representation = Latent_Representation.cpu().detach().numpy()

        ##################################################### Results  ####################################################


        if Clustering and (epoch + 1) % 5 == 0:
            print("Epoch:{},Loss:{:.4f}".format(epoch + 1, loss.item()))
            ACC_H2 = []
            NMI_H2 = []
            PUR_H2 = []
            ARI_H2 = []
    #         kmeans = KMeans(n_clusters=max(np.int_(Labels).flatten()))
#             kmeans = KMeans(n_clusters=label_sze)
    #         print(type(Latent_Representation))
    #         print(type(label))
            label = np.array(label1)
    #         print(type(label_sze))
    #         StandardScaler().fit_transform(label['label’].values.reshape(-1, 1))
            ari, nmi = cluster(Latent_Representation, label, label_sze)
            ari_max = 0
            nmi_max = 0
            if(ari>ari_max):
                ari_max = ari
            if(nmi>nmi_max):
                nmi_max = nmi
#             for i in range(10):
#                 Y_pred_OK = kmeans.fit_predict(Latent_Representation)     
#                 Labels_K = np.array(Labels).flatten()
#                 ARI = metrics.adjusted_rand_score(Y_pred_OK, Labels_K) 
#                 AM = clustering_metrics(Y_pred_OK, Labels_K)
#                 ACC, NMI, PUR = AM.evaluationClusterModelFromLabel(print_msg=False)
#                 ACC_H2.append(ACC)
#                 NMI_H2.append(NMI)
#                 PUR_H2.append(PUR)
#                 ARI_H2.append(ARI)

#             ACC_GAE_total.append(100 * np.mean(ACC_H2))
#             NMI_GAE_total.append(100 * np.mean(NMI_H2))
#             PUR_GAE_total.append(100 * np.mean(PUR_H2))
#             ARI_GAE_total.append(100 * np.mean(ARI_H2))
#             ACC_GAE_total_STD.append(100 * np.std(ACC_H2))
#             NMI_GAE_total_STD.append(100 * np.std(NMI_H2))
#             PUR_GAE_total_STD.append(100 * np.std(PUR_H2))
#             ARI_GAE_total_STD.append(100 * np.std(ARI_H2))
            
#             Index_MAX = np.argmax(ari_max)
#             ACC_GAE_max = np.float(ACC_GAE_total[Index_MAX])
#             NMI_GAE_max = np.float(NMI_GAE_total[Index_MAX])
#             PUR_GAE_max = np.float(PUR_GAE_total[Index_MAX])
#             ARI_GAE_max = np.float(ARI_GAE_total[Index_MAX])

#             print('ACC_H2=', 100 * np.mean(ACC_H2), '\n', 'ARI_H2=', 100 * np.mean(ARI_H2), '\n','NMI_H2=', 100 * np.mean(NMI_H2), '\n', 'PUR_H2=',
#                   100 * np.mean(PUR_H2))

            np.save(path_result + "{}.npy".format(epoch + 1), Latent_Representation)

    ###############################################################Clustering  Result ##############################
    if Clustering:
        Index_MAX = np.argmax(ari_max)
#         ACC_GAE_max = np.float(ACC_GAE_total[Index_MAX])
#         NMI_GAE_max = np.float(NMI_GAE_total[Index_MAX])
#         PUR_GAE_max = np.float(PUR_GAE_total[Index_MAX])
#         ARI_GAE_max = np.float(ARI_GAE_total[Index_MAX])

#         ACC_STD = np.float(ACC_GAE_total_STD[Index_MAX])
#         NMI_STD = np.float(NMI_GAE_total_STD[Index_MAX])
#         PUR_STD = np.float(PUR_GAE_total_STD[Index_MAX])
#         ARI_STD = np.float(ARI_GAE_total_STD[Index_MAX])
        print('ari_max={}' .format(ari_max))
        print('nmi_max={}' .format(nmi_max))
#         print('ACC_GAE_max={:.2f} +- {:.2f}'.format(ACC_GAE_max, ACC_STD))
#         print('NMI_GAE_max={:.2f} +- {:.2f}'.format(NMI_GAE_max, NMI_STD))
#         print('PUR_GAE_max={:.2f} +- {:.2f}'.format(PUR_GAE_max, PUR_STD))
#         print('ARI_GAE_max={:.2f} +- {:.2f}'.format(ARI_GAE_max, ARI_STD))
#         print("The incompleteness of the adjacency matrix is {}%".format(scale * 100))

    ########################################################### t- SNE #################################################
    if t_SNE:
        print("Index_Max = {}".format(Index_MAX))
        Latent_Representation_max = np.load(path_result + "{}.npy".format((Index_MAX+1) * 5))
        Features = np.array(Features)
#         print(type(Labels))
#         print(Labels.shape)
#         print(Labels)
#         print(type(Features))
#         print(Features.shape)
#         print(Features)
        plot_embeddings(Latent_Representation_max, Features, Labels)
        savefig(r'C:\Users\86166\Desktop\2022.4.25实验结果\聚类图\pca1\{}PCA111.jpg'.format(name))
        plt.show()
        plot_embedding(Latent_Representation_max, Features, Labels)
        savefig(r'C:\Users\86166\Desktop\2022.4.25实验结果\聚类图\t-sne1\{}T-SNE111.jpg'.format(name))
        plt.show()
    ########################################################################################################################
#     end_time = time.time()
#     ARI_GAE_max = ARI_GAE_max/100
#     NMI_GAE_max = NMI_GAE_max/100
    end_time = time.time()
    print("Running time is {}".format(end_time - start_time))
    return (ari_max,nmi_max)




from sklearn.datasets import make_moons
from sklearn.metrics import silhouette_score 
import sys
sys.path.append('./Data_Process')
path_result = "./Latent_representation/"
# from Models import *
from Metrics import *
from sklearn import metrics
import scipy.io as scio
from Data_Process import *
from sklearn.cluster import KMeans
from matplotlib.pyplot import plot,savefig
# from pandas_datareader import data, wb
import numpy as np
import torch
import time
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


######################################################### Setting #####################################################
Clustering = True
t_SNE = True
scale = 0
########################################## hyper-parameters##############################################################
Epoch_Num = 10
Learning_Rate = 1e-4

Hidden_Layer_1 = 800
# Hidden_Layer_2 = 50

################################### Load dataset   ######################################################################

if __name__ == '__main__':
    #定义一个DF，记录指标得分
    record = pd.DataFrame(
        columns=['名称', 'x', 'y', '类别数','层次聚类 ARI','层次聚类 NMI', '运行时间'])
    for name in data_names[:]:
        print(name)
        start = time.perf_counter() 
        #读取数据
        
        data = pd.read_csv(r'D:\scRNA数据集\data0317\data20220307\{}.csv'.format(name)).iloc[:,1:]
#         data = pd.read_csv(r'E:\data20220307\data20220307\{}.csv'.format(name)).iloc[:,1:]
        data = pd.DataFrame(data)
        # #分离特征和标签
        X1 = data.drop('label', axis=1)
        #label1 = data['label'].values
        label=pd.Categorical(data.iloc[:,-1]).codes
        
         #记录原始维度
        y_shape = X1.shape[1]
        x_shape = X1.shape[0]
        label_sze = len(np.unique(label))
        data1 = pd.DataFrame(X1)
        data = del_zero(data1)
        data_ol1 = points_to_the_noise(data,bins = 20,std_ol=4)
        data_ol = data_standardization(data_ol1)
#         X = DataFrame.as_matrix(data_ol)
        X = data_ol.values
        get_ann(X.shape[0],X.shape[1])  
        new_labels = labels
        data_ol = data_ol.T
        data_ol1 = pd.DataFrame(np.array(data_ol),columns=['{}'.format(i) for i in range(data_ol.shape[1])])
        data_ol = pd.DataFrame(np.array(data_ol),columns=['cell{}'.format(i) for i in range(data_ol.shape[1])])
        B = creat_adjmatrix(new_labels)
        data = pd.DataFrame(B)
        Adjacency_Matrix_raw = B
        Features, Labels = loaddata(data_ol1,label)
#         print(Features.is_cuda)
#         print(Labels)
        ari, nmi = graph_autoencoder(Features, Labels, Adjacency_Matrix_raw,label)
        end = time.perf_counter() 
        print(ari, nmi)
        score = [name, x_shape, y_shape, label_sze, ari, nmi, end - start]
        record.loc['{}'.format(name)] = score
#         record.to_csv('./实验结果2022.04.25层次聚类.csv')
        record.to_csv(r'C:/Users/86166/Desktop/2022.4.25实验结果/评估指标/实验结果2022.04.25.csv')

        
        
        
        


Kumar
